## Importing Boston Marathon data

In this notebook, we'll read in the Boston Marathon data from 2013-17.

The official results can be found __[here](http://registration.baa.org/cfm_Archive/iframe_ArchiveSearch.cfm)__. However, this link doesn't show split times. One can find official splits for __2017__ at the following site --

http://registration.baa.org/2017/cf/public/iframe_ResultsSearch.cfm

(From the URL, one can infer the web addresses for 2013-6.)

I scraped the data and included them as CSV files in the same repository as this notebook file.

Here, we'll import the data files, calculate a lot of pacing metrics, and combine the expanded dataframes into a giant *boscomplete* dataframe. Then we'll write out *boscomplete* to a CSV file.

In [1]:
# importing modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from datetime import datetime
import datetime as dt
#from helperFunctions import BQlookup2, get_sec, get_time, getDivision
from __future__ import print_function
pd.set_option('max_colwidth',120)

In [2]:
# timing mat info
halfway = 'split5_time'

marDist = 26.21875 # marathon distance in miles
fiveK = 3.106856 # 5K distance in miles
mats = [fiveK, fiveK*2, fiveK*3, fiveK*4, 0.5*marDist, fiveK*5, fiveK*6, fiveK*7, fiveK*8, marDist] # comprehensive
timingmats = {i+1:mats[i] for i in range(len(mats))}
# dictionary: timingmats[1] = 5K, timingmats[2] = 10K, ... , timingmats[10] = marDist

In [3]:
def get_sec(s):
# takes in text string 'HH:MM:SS.mss' and returns time in seconds
# can also be in MM:SS form
    import numpy as np

    try:
        if s == 'DNF':
            return np.NaN
        elif s == 'DSQ':
            return np.NaN
        elif s == 'DQ':
            return np.NaN
        elif s == 'N/A':
            return np.NaN
        elif s == 'nan':
            return np.NaN
        elif len(s) < 3:
            return np.NaN
        else:
            l = s.split(':')
            if len(l) == 3:
                if l[0][0] != '-':
                    return int(l[0]) * 3600 + int(l[1]) * 60 + int(float(l[2]))
                else: # negative!
                    return int(l[0]) * 3600 - int(l[1]) * 60 - int(float(l[2]))
        #needed to deal with the case of milliseconds in finish time: truncate to seconds
            elif len(l) == 2:
                if l[0][0] != '-':
                    return int(l[0]) * 60 + int(float(l[1]))
                else: # negative!
                    return int(l[0]) * 60 - int(float(l[1]))
            else:
                return "bad format!!"
    except TypeError:
        # this catches NaN inputs when "len(s) < 3" is checked
        return np.NaN

def get_time(sec):
# takes in number (can be a decimal) and returns text string '[H]H:MM:SS'
# or '[M]M:SS' if less than 1 hour
    import math
    import numpy as np
    
    if sec == '':
        return ''
    elif np.isnan(sec) == True:
        return ''
    
    if sec < 0:
        sign = "-"
    else:
        sign = ""
    
    sec = round(sec) # added on 8/30/2016 T

    if abs(sec) >= 60*60: # if the time is greater than one hour
        hours = int(math.floor(abs(sec) / 3600))
        minutes = int(math.floor((abs(sec)-hours*3600) / 60))
        if minutes < 10:
            minutes = '0' + str(int(minutes))
        else:
            minutes = str(int(minutes))
        seconds = abs(sec) - hours*3600 - int(minutes)*60
        if seconds < 10:
            seconds = '0' + str(int(seconds))
        else:
            seconds = str(int(seconds))
        return sign + str(hours) + ":" + minutes + ":" + seconds
    elif abs(sec) < 60*60 and abs(sec) >= 60:
        # if the time is less than 1 hour but greater than 1 minute
        minutes = int(math.floor(abs(sec) / 60))
#        if minutes < 10:
#            minutes = '0' + str(int(minutes))
#        else:
#            minutes = str(int(minutes))
        seconds = abs(sec) - int(minutes)*60
        if seconds < 10:
            seconds = '0' + str(int(seconds))
        else:
            seconds = str(int(seconds))
        return sign + str(minutes) + ":" + seconds
    elif abs(sec) < 60:
        seconds = abs(int(sec))
        if seconds < 10:
            seconds = '0' + str(int(seconds))
        else:
            seconds = str(int(seconds))
        return sign + "0:" + seconds

def get_time_formal(sec):
# takes in number (can be a decimal) and returns text string 'H:MM:SS'

    import math
    import numpy as np
    import datetime as dt
    
    if sec == '':
        return ''
    elif np.isnan(sec) == True:
        return ''
    
    if sec < 0:
        sign = "-"
    else:
        sign = ""
    
    sec = round(sec)

    return sign + str(dt.timedelta(seconds=abs(sec)))

def BQlookup2(gender, age):
    # gender & age are scalars
    if gender == "NA":
        BQreq = "?"
    elif age == "NA":
        BQreq = "?"
  
    if gender == 'M':
        if age < 35:
            BQreq = "3:05:00"
        elif age < 40:
            BQreq = "3:10:00"
        elif age < 45:
            BQreq = "3:15:00"
        elif age < 50:
            BQreq = "3:25:00"
        elif age < 55:
            BQreq = "3:30:00"
        elif age < 60:
            BQreq = "3:40:00"
        elif age < 65:
            BQreq = "3:55:00"
        elif age < 70:
            BQreq = "4:10:00"
        elif age < 75:
            BQreq = "4:25:00"
        elif age < 80:
            BQreq = "4:40:00"
        else:
            BQreq = "4:55:00"
    
    elif (gender == 'F'):
        if (age < 35):
            BQreq = "3:35:00"
        elif (age < 40):
            BQreq = "3:40:00"
        elif (age < 45):
            BQreq = "3:45:00"
        elif (age < 50):
            BQreq = "3:55:00"
        elif (age < 55):
            BQreq = "4:00:00"
        elif (age < 60):
            BQreq = "4:10:00"
        elif (age < 65):
            BQreq = "4:25:00"
        elif (age < 70):
            BQreq = "4:40:00"
        elif (age < 75):
            BQreq = "4:55:00"
        elif (age < 80):
            BQreq = "5:10:00"
        else:
            BQreq = "5:25:00"

    return(BQreq)

def getDivision(gender,age):
    if age < 20:
        age_str = '0_19'
    elif age < 25:
        age_str = '20_24'
    elif age < 30:
        age_str = '25_29'
    elif age < 35:
        age_str = '30_34'
    elif age < 40:
        age_str = '35_39'
    elif age < 45:
        age_str = '40_44'
    elif age < 50:
        age_str = '45_49'
    elif age < 55:
        age_str = '50_54'
    elif age < 60:
        age_str = '55_59'
    elif age < 65:
        age_str = '60_64'
    elif age < 70:
        age_str = '65_69'
    elif age < 75:
        age_str = '70_74'
    elif age < 80:
        age_str = '75_79'
    else:
        age_str = '80+'
    return (gender + age_str)

def addColumns2(full_DF, time_col, gender_col, age_col, age_up, mats):
    import matplotlib.dates as mdates
    import unicodedata
    
    timingmats = {i+1:mats[i] for i in range(len(mats))}
        
    full_DF.rename(columns = {'finish_guntime': 'gun',
                          'finish_time': 'chip',
                          'finish_chiptime': 'chip',
                          'chip_time': 'chip',
                          'place_y': 'place'}, inplace=True)
    full_DF = full_DF[full_DF.chip.notnull()]
    full_DF = full_DF[~full_DF.chip.str.isspace()]
    full_DF = full_DF[~full_DF.chip.isin(["DNF","-DNF-","DNS","DQ","DSQ","00:00:00","0:00:00"])]
    full_DF.reset_index(drop=True,inplace=True)
    full_DF['name'].replace(np.nan,u'UNKNOWN',inplace=True)
    #full_DF['name'] = full_DF.apply(lambda row: unicodedata.normalize('NFD', row['name']).encode('ascii', 'ignore'),axis=1)
    full_DF['name'] = full_DF.name.str.replace("\.","").str.strip()
    full_DF.columns = [x.replace("_chiptime","_time") for x in full_DF.columns]

    findhalf = [abs(x - 0.5*marDist) for x in mats]
    halfway_index = (findhalf.index(min(findhalf))) # 1 if 'split2_time', 2 if 'split3_time', etc
    halfway = "split" + str(halfway_index+1) + "_time"
    findtenK = [abs(x - fiveK*2) for x in mats]
    tenK_index = (findtenK.index(min(findtenK)))
    tenK = "split" + str(tenK_index+1) + "_time"
    
    for x in [('split'+str(y)+'_time') for y in range(1,len(mats))]:
        full_DF[x].replace("00:00:00","",inplace=True)
        full_DF[x].replace("-","",inplace=True)
        full_DF[x].replace(regex=r'\s+',value="",inplace=True)
        full_DF[x].replace(np.nan,"",inplace=True)
        
    full_DF['seconds'] = full_DF.apply(lambda row: get_sec(row[time_col]), axis=1)
    full_DF[gender_col] = full_DF.apply(lambda row: 'M' if row[gender_col][0] == 'M'
                                       else 'F', axis=1)

    full_DF['sex_int'] = full_DF.apply(lambda row: 0 if row[gender_col]=="M" else 1,axis=1)
    
    if age_up == True:
        full_DF['BQreq'] = full_DF.apply(lambda row: BQlookup2(row[gender_col],row[age_col]+1),axis=1)
    else:
        full_DF['BQreq'] = full_DF.apply(lambda row: BQlookup2(row[gender_col],row[age_col]),axis=1)
    full_DF['BQ']= full_DF.apply(lambda row: 1 if ( (row['seconds'] <= get_sec(row['BQreq']) ) and row[time_col][0:4] != '0:00')
                                            else 0, axis=1)
    #needed to make sure "0:00" finish times don't get counted as BQs!!
    full_DF['DIVISION'] = full_DF.apply(lambda row: getDivision(row[gender_col],row[age_col]), axis=1)
    full_DF['epoch'] = mdates.epoch2num(full_DF['seconds'])

    #return full_DF
    full_DF['BQpace_sec'] = full_DF.apply(lambda row: get_sec(row['BQreq'])*1.0/marDist, axis=1) # decimal, not int
    full_DF['BQpace'] = full_DF['BQpace_sec'].apply(get_time) # MM:SS format, rounds to nearest second

    # whether paces are provided or calculated, I want missing paces to show up as "", not np.nan !!
    # this is because, when creating the 'mm' column, the algorithm counts # of "" pace entries
    # Calculate paces & split paces --
    for x in range(1,len(timingmats)):
        y = 'split' + str(x) + '_pace'
        mat = 'split' + str(x) + '_time'
        score = 'split' + str(x) + '_score'
        full_DF[y] = (full_DF[mat].apply(get_sec) * 1.0/timingmats[x] ).apply(get_time)
        full_DF[score] = (full_DF[mat].apply(get_sec) * 1.0/timingmats[x] ) - full_DF['BQpace_sec']

        splitpace = 'split' + str(x) + str(x+1) + '_pace'
        if x == (len(timingmats)-1):
            nextmat = 'chip'
        else:
            nextmat = 'split' + str(x+1) + '_time'
        full_DF[splitpace] = ((full_DF[nextmat].apply(get_sec) - full_DF[mat].apply(get_sec))*1.0/(timingmats[x+1]-timingmats[x])).apply(get_time)
        splitpacescore = 'split' + str(x) + str(x+1) + '_score'
        full_DF[splitpacescore] = ((full_DF[nextmat].apply(get_sec) - full_DF[mat].apply(get_sec))*1.0/(timingmats[x+1]-timingmats[x])) - full_DF['BQpace_sec']

    full_DF['pace'] = (full_DF['chip'].apply(get_sec) * 1.0/marDist ).apply(get_time)
    full_DF['BQdiff'] = full_DF['seconds'] - full_DF['BQreq'].apply(get_sec) #negative values means BQed

    full_DF['2ndhalf'] = (full_DF['chip'].apply(get_sec) - full_DF[halfway].apply(get_sec)).apply(get_time)
    full_DF['2ndhalfpace'] = (full_DF['2ndhalf'].apply(get_sec)*1.0/(marDist-timingmats[halfway_index+1]))#.apply(get_time)
    full_DF['1sthalfpace'] = (full_DF[halfway].apply(get_sec)*1.0/(timingmats[halfway_index+1]))#.apply(get_time)
    full_DF['pace_diff'] = full_DF['2ndhalfpace'] - full_DF['1sthalfpace'] # negative means negative split
    full_DF['splits'] = full_DF['2ndhalf'].apply(get_sec) - full_DF[halfway].apply(get_sec)

    print("{} records".format(len(full_DF)))
    splt = "[get_sec(row.split1_pace)"
    pace = "[get_sec(row.split1_pace)"
    time = "[get_sec(row.split1_time)"
    mm = "[row.split1_pace"
    for x in range(1,len(timingmats)):
        splt = splt + ",get_sec(row.split" + str(x) + str(x+1) + "_pace)"
        if x == 1:
            continue
        else:
            pace = pace + ",get_sec(row.split" + str(x) + "_pace)"
            time = time + ",get_sec(row.split" + str(x) + "_time)"
            mm = mm + ",row.split" + str(x) + "_pace"
    splt = "full_DF.apply(lambda row: " + splt + "], axis=1)"
    pace = "full_DF.apply(lambda row: " + pace + ",get_sec(row.pace)], axis=1)"
    time = "full_DF.apply(lambda row: " + time + ",get_sec(row.chip)], axis=1)"
    mm = "full_DF.apply(lambda row: [(i+1) for i,j in enumerate(" + mm + ",row.pace]) if j==''], axis=1)"

    full_DF['mm'] = eval(mm)
    full_DF['mm_len'] = full_DF['mm'].apply(len)

    full_DF['splitpaces'] = eval(splt)
    full_DF['cumpaces'] = eval(pace)
    full_DF['allsplits'] = eval(time)

    full_DF['non_missing_mats'] = full_DF.apply(lambda row: sorted(list( set(range(1,len(timingmats)+1)) - set(row['mm']) )),
                                             axis=1)
    full_DF['non_missing_cumpaces'] = full_DF.apply(lambda row: [ row['cumpaces'][y-1] for y in row['non_missing_mats'] ],
                                             axis=1)
    full_DF['non_missing_times'] = full_DF.apply(lambda row: [ row['allsplits'][y-1] for y in row['non_missing_mats'] ],
                                             axis=1)
    full_DF['non_missing_splitpaces'] = full_DF.apply(lambda row: 
        [(1.0*row['non_missing_times'][0]/timingmats[row['non_missing_mats'][0]])] + 
        [(1.0*(row['non_missing_times'][x+1]-row['non_missing_times'][x])/(timingmats[row['non_missing_mats'][x+1]]-timingmats[row['non_missing_mats'][x]])) 
        for x in range(len(row['non_missing_mats'])-1)], axis=1)
    full_DF['non_missing_splitpaces_time'] = full_DF.apply(lambda row: map(get_time, row['non_missing_splitpaces']), axis=1)
    full_DF['stdpace'] = full_DF['non_missing_splitpaces'].apply(np.nanstd,ddof=1)

    print("{} (median stdpace)".format(full_DF['stdpace'].median()))
    print("{} (median splits)".format(get_time(full_DF['splits'].median())))

    return full_DF

In [4]:
print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))

print("\n2013")
bos13 = pd.read_csv("2013bostonofficial_NOV2017.csv")
#bos13 = addColumns2(bos13, 'chip','sex','age', True, mats) # age_up is 5th input

print("\n2014")
bos14 = pd.read_csv("2014bostonofficial_NOV2017.csv")
bos14['chip'] = bos14.chip.str.replace("\.",":") # need this to fix Timothy Knott's chip time
#bos14 = addColumns2(bos14, 'chip','sex','age', True, mats) # age_up is 5th input

print("\n2015")
bos15 = pd.read_csv("2015bostonofficial_NOV2017.csv")
#bos15 = addColumns2(bos15, 'chip','sex','age', True, mats) # age_up is 5th input

print("\n2016")
bos16 = pd.read_csv("2016bostonofficial_NOV2017.csv")
#bos16 = addColumns2(bos16, 'chip','sex','age', True, mats) # age_up is 5th input

print("\n2017")
bos17 = pd.read_csv("2017bostonofficial_NOV2017.csv")
#bos17 = addColumns2(bos17, 'chip','sex','age', True, mats) # age_up is 5th input

print("")
print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))

Friday, 02 March 2018 12:39:43PM

2013

2014

2015

2016

2017

Friday, 02 March 2018 12:39:44PM


In [5]:
# WEATHER in Hopkinton, MA on race day, at or close to 10 AM
# https://www.wunderground.com/history/

bos13['start_temp'] = 50.0
bos14['start_temp'] = 52.0
bos15['start_temp'] = 45.0
bos16['start_temp'] = 66.9
bos17['start_temp'] = 72.0

bos13['year'] = 2013
bos14['year'] = 2014
bos15['year'] = 2015
bos16['year'] = 2016
bos17['year'] = 2017

# wave info can be found at --
# http://registration.baa.org/201X/cf/Public/iframe_EntryLists.cfm
bos13['wave'] = bos13.apply(lambda row: 0 if "F" in row.bib
                                        else 0 if int(row.bib)<=100
                                        else 1 if (int(row.bib)<9000)
                                        else 2 if (int(row.bib)<18000)
                                        else 3, axis=1)
bos14['wave'] = bos14.apply(lambda row: 0 if "F" in row.bib
                                        else 0 if int(row.bib)<=100
                                        else 1 if (int(row.bib)<9000)
                                        else 2 if (int(row.bib)<18000)
                                        else 3 if (int(row.bib)<27000)
                                        else 4, axis=1)
bos15['wave'] = bos15.apply(lambda row: 0 if "F" in row.bib
                                        else 0 if int(row.bib)<=100
                                        else 1 if (int(row.bib)<8000)
                                        else 2 if (int(row.bib)<16000)
                                        else 3 if (int(row.bib)<24000)
                                        else 4, axis=1)
bos16['wave'] = bos16.apply(lambda row: 0 if "F" in row.bib
                                        else 0 if int(row.bib)<=100
                                        else 1 if (int(row.bib)<8000)
                                        else 2 if (int(row.bib)<16000)
                                        else 3 if (int(row.bib)<24000)
                                        else 4, axis=1)
bos17['wave'] = bos17.apply(lambda row: 0 if "F" in row.bib
                                        else 0 if int(row.bib)<=100
                                        else 1 if (int(row.bib)<8000)
                                        else 2 if (int(row.bib)<16000)
                                        else 3 if (int(row.bib)<24000)
                                        else 4, axis=1)

In [6]:
# load unofficial results, to get info that's not in official results (gun time, unofficial chip time)
# (it's important to know if someone's official chip time is different than her unofficial chip time)

det13 = pd.read_csv("2013bostonunofficial.csv",low_memory=False)
det14 = pd.read_csv("2014bostonunofficial.csv",low_memory=False)
det15 = pd.read_csv("2015bostonunofficial.csv",low_memory=False)
det16 = pd.read_csv("2016bostonunofficial.csv",low_memory=False)
det17 = pd.read_csv("2017bostonunofficial.csv",low_memory=False)

for df in [det13, det14, det15, det16, det17]:
    # replace anything that's not HH:MM:SS with ""
    for col in [('split'+str(y)+'_time') for y in range(1,len(mats))] + ['chip','gun']:
        df[col].replace("-","",inplace=True)
        df[col].replace(regex=r'\s+',value="",inplace=True)
        df[col].replace(np.nan,"",inplace=True)
    # get rid of leading "0" in chip & gun (change from HH:MM:SS to H:MM:SS)
    df.chip = df.chip.str.replace("^0","")
    df.gun = df.gun.str.replace("^0","")

# the unofficial results have different bib numbers for a few people
# we'll change those bib numbers to what's shown in the official results

det13.bib.replace('14995','10735',inplace=True) # Floss, Peter
det13.bib.replace('27141','24585',inplace=True) # Dugan, Timothy
det14.bib.replace('4994','2281',inplace=True) # Muhlemann Bk, Stefan
det14.bib.replace('4995','7024',inplace=True) # Miller, Justin
det14.bib.replace('9994','12900',inplace=True) # Schroeder, Karen
det14.bib.replace('35995','19257',inplace=True) # Kresky-Griffin, Lisa
det14.bib.replace('15992','15940',inplace=True) # Park, Hyunsoo
det14.bib.replace('19996','19066',inplace=True) # Holmes, Carol
det15.bib.replace('5997','5368',inplace=True) # Bell, David
det15.bib.replace('5994','3469',inplace=True) # Braunshausen, Andy
det15.bib.replace('13995','14840',inplace=True) # Dowd, Bridget
det15.bib.replace('5996','3136',inplace=True) # Hallgren, Krister
det15.bib.replace('21995','20399',inplace=True) # Harris, Rebecca
det15.bib.replace('13994','14381',inplace=True) # Kenutis, Sarah
det15.bib.replace('21994','18447',inplace=True) # Kreutz, Wendi
det15.bib.replace('21993','19783',inplace=True) # Marshall, Julie
det15.bib.replace('25994','29027',inplace=True) # Sheeleigh, Matthias
det15.bib.replace('5995','4411',inplace=True) # Suarez Arias, Miguel
det15.bib.replace('25995','24077',inplace=True) # Williamson, Delwyn
det16.bib.replace('22990','19399',inplace=True) # Altendorf, Timothy
det16.bib.replace('14990','8226',inplace=True) # Calderon, Ramiro
det16.bib.replace('15585','15144',inplace=True) # Fox, John
det16.bib.replace('32114','30334',inplace=True) # Francis, David
det16.bib.replace('4998','3945',inplace=True) # Kositzke, Jacob
det16.bib.replace('23599','25448',inplace=True) # Pascua, Luigi
det16.bib.replace('19993','19390',inplace=True) # Spence, Steve
det16.bib.replace('32115','31323',inplace=True) # Yagoda, Aaron
det16.bib.replace('19992','17209',inplace=True) # Yingling, Leah
print("finished!")

finished!


In [7]:
try:
    del bos13['gun']
    del bos14['gun']
    del bos15['gun']
    del bos16['gun']
    del bos17['gun']
except:
    print("nothing to delete!")
    
# merge official & unofficial results (to get 'gun' & 'chip' columns from unofficial results)
mer13 = pd.merge(bos13, det13, on='bib')
print(len(bos13), len(det13), len(mer13))
mer14 = pd.merge(bos14, det14, on='bib')
print(len(bos14), len(det14), len(mer14))
mer15 = pd.merge(bos15, det15, on='bib')
print(len(bos15), len(det15), len(mer15))
mer16 = pd.merge(bos16, det16, on='bib')
print(len(bos16), len(det16), len(mer16))
mer17 = pd.merge(bos17, det17, on='bib')
print(len(bos17), len(det17), len(mer17))

# get 'gun' column from unofficial results (official results don't have 'gun' column)
bos13['gun'] = mer13['gun']
bos14['gun'] = mer14['gun']
bos15['gun'] = mer15['gun']
bos16['gun'] = mer16['gun']
bos17['gun'] = mer17['gun']

# get 'chip' column from unofficial results
bos13['chip_unofficial'] = mer13['chip_y']
bos14['chip_unofficial'] = mer14['chip_y']
bos15['chip_unofficial'] = mer15['chip_y']
bos16['chip_unofficial'] = mer16['chip_y']
bos17['chip_unofficial'] = mer17['chip_y']

nothing to delete!
17600 26839 17600
31924 35671 31924
26598 30252 26598
26630 30743 26630
26401 30074 26401


In [8]:
savethis = pd.concat([bos13,bos14,bos15,bos16,bos17])

print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))
savethis = addColumns2(savethis, 'chip','sex','age', True, mats) # age_up is 5th input
savethis['bib'] = savethis.bib.apply(str)
savethis['sex_int'] = savethis.apply(lambda row: 0 if row.sex=="M" else 1,axis=1)
print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))

Friday, 02 March 2018 12:40:03PM
129153 records
35.6076523116 (median stdpace)
11:30 (median splits)
Friday, 02 March 2018 01:22:17PM


/Users/donwang/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1147: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


### Data wrangling & cleaning

The data importing process involves an important step: making sure that the data are in a good format, and, if not, to clean the bad parts.

Here, we'll show that there are cases where split times need to be adjusted due to differences between official & unofficial chip times (i.e., chip!=chip_unofficial). (Split times will need to be adjusted in cases where a runner's first & last split paces don't make sense.)

In [9]:
# find people with sub-4:20 split anywhere along course
df = savethis.copy()
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','stdpace','mm','non_missing_splitpaces_time']
anomaly1 = df[(df.apply(lambda row: (min(row['non_missing_splitpaces'])<=60*4.33),axis=1)==1)]
print(len(anomaly1))
(anomaly1[cols])

29


,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time
19743,2014,1,7487,"Bertram, Troy M",M40_44,2:58:59,3:04:40,3:04:40,91.665057,[],"[8:36, 6:49, 6:39, 6:46, 6:42, 6:49, 6:52, 7:05, 6:57, 2:33]"
19996,2014,1,7235,"Toudal, Steen K",M40_44,3:00:08,3:05:32,3:05:32,85.884533,[],"[8:35, 6:50, 6:48, 6:53, 6:37, 6:52, 7:02, 6:58, 6:48, 2:52]"
20445,2014,1,6380,"Spouse, Antony J",M40_44,3:02:52,3:07:08,3:07:08,62.211020,[],"[8:22, 7:09, 6:44, 6:51, 6:43, 6:53, 6:57, 6:55, 7:12, 4:11]"
21357,2014,1,6902,"Smith, Matthew C",M35_39,3:08:03,3:12:47,3:12:47,68.897363,[],"[8:38, 7:06, 6:56, 7:01, 6:54, 6:59, 7:12, 7:25, 7:31, 4:03]"
22435,2014,1,7901,"Gilreath, James R Jr",M40_44,3:13:06,3:18:29,3:18:29,71.394755,[],"[8:48, 7:01, 7:02, 7:07, 7:13, 7:15, 7:28, 7:45, 7:54, 4:09]"
22598,2014,0,311,"Quednau, Todd",M30_34,3:13:46,3:19:26,3:19:26,92.632960,[],"[8:48, 6:57, 6:56, 7:12, 7:01, 7:19, 8:15, 7:45, 7:53, 3:05]"
25761,2014,0,8133,"Davenport, Michael J",M45_49,3:26:17,3:33:41,3:33:41,94.262073,[],"[9:29, 7:10, 7:13, 7:24, 7:20, 7:29, 8:08, 8:53, 8:59, 3:49]"
27544,2014,1,12578,"Karl, Lou",M55_59,3:32:00,3:44:00,3:44:00,155.274567,[],"[11:46, 7:46, 7:51, 8:00, 8:16, 7:46, 8:05, 8:01, 8:21, 1:10]"
29177,2014,1,19290,"Vanore, Marialuisa",F40_44,3:37:20,3:44:42,3:45:13,105.336663,[],"[7:14, 7:33, 8:08, 8:36, 8:30, 8:54, 9:11, 9:40, 9:14, 3:33]"
31244,2014,0,15089,"Potts, Franzine B",F30_34,3:44:06,4:54:28,5:07:28,961.337157,[],"[7:48, 11:16, 10:24, 10:50, 9:33, 11:41, 11:46, 13:12, 12:56, -39:22]"


In [10]:
# BQ cases (not necessarily with sub-4:20 split pace) -- some of these appear above in anomaly1
# split1_pace is too slow relative to split12_pace
# split89_pace is too fast relative to split910_pace
df = savethis.copy()
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','stdpace','mm','non_missing_splitpaces_time']
anomaly2 = df[df.BQ.isin([1]) 
    & (df.chip!=df.chip_unofficial)
    & (df.split1_score-df.split12_score>=30)
    & (df.split89_score-df.split910_score>=60)]
print(len(anomaly2))
(anomaly2.sort_values(by=['mm_len','stdpace'],ascending=False)[cols])

46


,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time
27544,2014,1,12578,"Karl, Lou",M55_59,3:32:00,3:44:00,3:44:00,155.274567,[],"[11:46, 7:46, 7:51, 8:00, 8:16, 7:46, 8:05, 8:01, 8:21, 1:10]"
52473,2015,1,30877,"Palmer, Andy",M30_34,3:01:34,3:08:40,3:08:40,122.776750,[],"[10:16, 6:47, 6:39, 6:52, 6:26, 6:38, 6:46, 6:50, 6:56, 1:43]"
55136,2015,1,7171,"Wong, Steven S",M40_44,3:14:13,3:19:21,3:19:21,95.590502,[],"[9:14, 7:27, 7:34, 8:14, 7:01, 7:01, 7:23, 7:22, 6:56, 3:03]"
19743,2014,1,7487,"Bertram, Troy M",M40_44,2:58:59,3:04:40,3:04:40,91.665057,[],"[8:36, 6:49, 6:39, 6:46, 6:42, 6:49, 6:52, 7:05, 6:57, 2:33]"
19996,2014,1,7235,"Toudal, Steen K",M40_44,3:00:08,3:05:32,3:05:32,85.884533,[],"[8:35, 6:50, 6:48, 6:53, 6:37, 6:52, 7:02, 6:58, 6:48, 2:52]"
56190,2015,1,14625,"Trout, Christina",F30_34,3:18:42,3:23:42,3:23:42,79.581068,[],"[9:00, 7:25, 7:29, 7:29, 7:29, 7:33, 7:44, 7:53, 7:45, 3:49]"
52574,2015,1,953,"Gyr, Kiran A",M35_39,3:02:05,3:06:38,3:06:38,78.101471,[],"[8:23, 7:01, 6:52, 6:56, 6:56, 6:59, 7:01, 7:05, 6:54, 3:16]"
32989,2014,1,21827,"Maus, Shannon C",F50_54,3:50:18,3:55:18,3:55:18,73.504319,[],"[10:13, 8:41, 8:34, 8:42, 8:35, 8:45, 8:55, 9:11, 8:47, 5:23]"
55588,2015,1,7213,"Ptacek, Lubor",M45_49,3:16:15,3:21:22,3:21:22,71.430005,[],"[8:52, 7:13, 7:13, 7:17, 7:17, 7:24, 7:43, 7:56, 7:43, 4:13]"
22435,2014,1,7901,"Gilreath, James R Jr",M40_44,3:13:06,3:18:29,3:18:29,71.394755,[],"[8:48, 7:01, 7:02, 7:07, 7:13, 7:15, 7:28, 7:45, 7:54, 4:09]"


In [11]:
# find chip times that don't make sense
df = savethis.copy()
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','stdpace','mm','non_missing_splitpaces_time']
anomaly3 = df[(df.apply(lambda row: (int(row['chip'][-5:-3])>59)
                                          or (int(row['chip'][-2:])>59),axis=1)==1)]
(anomaly3[cols])
# Because his chip time was adjusted (chip!=chip_unofficial), his split times and paces will need to be adjusted as well
# He also shows up in anomaly2 above due to his first & last split paces

,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time
51902,2015,1,3820,"Wright, Brian",M25_29,2:58:81,3:00:38,3:00:38,29.454644,[],"[7:20, 6:36, 6:56, 6:39, 6:43, 6:45, 7:05, 7:07, 6:50, 5:31]"


From the anomalyX cases above, it appears that there aren't too many cases from 2016-17 that need to be fixed, in terms of adjusting the split times downward by the same amount that unofficial chip times were adjusted.

And this is in fact the case: the BAA started correcting these discrepancies in 2016, but left the previous years' results as-is.

This means that we can skip over 2016-17 when we're searching for records that have chip!=chip_unofficial.

(There are also two strange non-BQ cases from 2014 that we'll leave alone. They are such that if we adjust the split times downward, their split1_times would be negative. See the first two rows in the following cell's output.)

In [12]:
# don't fix these! (Aside from the 2014 cases, each runner's split times appear correct)
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','mm','non_missing_splitpaces_time']
df = savethis.copy()
donotfix = df[(df.chip!=df.chip_unofficial) & ~df.chip_unofficial.isin([""])
              & ((df.year.isin([2014]) & df.bib.isin(['15089','26680']))
             | (df.year.isin([2016,2017])))]
donotfix[cols]

,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,mm,non_missing_splitpaces_time
31244,2014,0,15089,"Potts, Franzine B",F30_34,3:44:06,4:54:28,5:07:28,[],"[7:48, 11:16, 10:24, 10:50, 9:33, 11:41, 11:46, 13:12, 12:56, -39:22]"
48209,2014,0,26680,"Martinovich, Patricia",F45_49,5:50:00,6:08:13,6:16:06,[],"[10:05, 10:53, 11:57, 13:15, 18:14, 15:38, 15:22, 19:53, 14:30, 1:15]"
76167,2016,1,102,"Ayr, Jason M",M25_29,2:32:44,2:32:59,2:32:59,[],"[5:42, 5:42, 5:41, 5:41, 5:37, 5:41, 5:50, 5:56, 6:08, 6:26]"
76341,2016,1,1522,"Gennaro, Thomas R Jr",M25_29,2:43:37,2:44:18,2:44:18,[],"[6:13, 6:03, 6:04, 6:10, 6:07, 6:17, 6:20, 6:28, 6:22, 6:15]"
76346,2016,1,2835,"Buell, Jon",M30_34,2:43:51,2:45:34,2:45:34,[],"[5:36, 5:58, 6:04, 6:13, 6:07, 6:14, 6:29, 6:40, 6:39, 6:34]"
76387,2016,1,581,"Phares, Marshall",M25_29,2:45:14,2:45:38,2:45:38,[],"[6:06, 6:12, 6:11, 6:11, 6:10, 6:12, 6:15, 6:25, 6:33, 7:05]"
76404,2016,1,1658,"Cipriano, Robert",M55_59,2:45:29,2:46:09,2:46:09,[],"[6:20, 6:08, 6:13, 6:15, 6:15, 6:14, 6:20, 6:38, 6:29, 6:01]"
76508,2016,1,1952,"Rodrigues, Vitor",M30_34,2:48:18,2:48:48,2:48:48,[],"[6:19, 6:13, 6:14, 6:22, 6:20, 6:24, 6:37, 6:44, 6:31, 6:24]"
76541,2016,1,893,"Nishimura, Akira",M40_44,2:48:57,2:49:07,2:49:07,[],"[6:01, 6:07, 6:19, 6:22, 6:18, 6:26, 6:37, 6:45, 6:43, 7:02]"
76562,2016,1,2967,"Bourque, Yves",M40_44,2:49:18,2:50:14,2:50:14,[],"[6:25, 6:16, 6:19, 6:23, 6:12, 6:19, 6:34, 6:43, 6:35, 6:47]"


In [13]:
# Fix split times and re-calculate all metrics for those with split times that need fixing
# Get a clean copy of boscomplete df

df = savethis.copy()
# find people with chip!=chip_unofficial and chip_unofficial!=""
indexes_to_fix = df[(df.chip!=df.chip_unofficial) & ~df.chip_unofficial.isin([''])].index
# don't fix these --
# 2014: 15089 (Potts), 26680 (Martinovich)
# all of 2016-17
donotfix = df[((df.year.isin([2014]) & df.bib.isin(['15089','26680']))
             | (df.year.isin([2016,2017]) ))]

indexes_to_fix = sorted(set(indexes_to_fix) - set(donotfix.index))

dffix = df.ix[indexes_to_fix]

# adjust all split times downward/faster by this amount: (chip_official - chip)
for col in [('split'+str(y)+'_time') for y in range(1,len(mats))]:
    dffix[col] = (dffix[col].apply(get_sec) - (dffix.chip_unofficial.apply(get_sec)-dffix.chip.apply(get_sec))).apply(get_time_formal)
dffix = addColumns2(dffix, 'chip','sex','age', True, mats) # age_up is 5th input
# addColumns2 resets the index of dffix, so put back original indexes
dffix.index = indexes_to_fix

# remove the rows that have anomalies
df = df.ix[set(df.index) - set(indexes_to_fix)]
# add the corrected rows back in
df = pd.concat([df,dffix]).sort_index()
# get a clean copy of the corrected dataframe
boscomplete = df.copy()

print(len(df), len(dffix), len(indexes_to_fix))
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','split1_time','mm','non_missing_splitpaces_time']
dffix.sort_values(by=['mm_len','stdpace'],ascending=False)[cols]
dffix[cols]

156 records
26.4917423136 (median stdpace)
7:34 (median splits)
129153 156 156


,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,split1_time,mm,non_missing_splitpaces_time
1066,2013,1,849,"Hennessey, Charlie A",M50_54,2:52:57,2:53:12,2:53:12,0:18:58,[],"[6:06, 6:15, 6:22, 6:30, 6:23, 6:31, 6:39, 6:53, 7:08, 7:29]"
1379,2013,1,5599,"Ness, Erik C",M35_39,2:55:59,2:59:12,2:59:12,0:21:01,[],"[6:46, 6:43, 6:37, 6:38, 6:40, 6:38, 6:46, 6:46, 6:46, 6:47]"
3735,2013,1,6465,"Dabee, Christophe",M45_49,3:09:58,3:10:01,3:14:01,0:21:45,[],"[7:00, 7:04, 7:06, 7:09, 7:04, 7:07, 7:16, 7:26, 7:35, 7:48]"
4267,2013,1,4002,"Shugart, Rob C",M40_44,3:12:44,3:15:40,3:15:40,0:21:32,[],"[6:56, 6:52, 6:56, 7:01, 6:58, 8:08, 7:26, 7:53, 7:46, 7:32]"
4754,2013,1,9472,"Vicary, Bret P",M55_59,3:15:08,3:15:25,3:15:25,0:21:32,[],"[6:56, 7:08, 7:16, 7:21, 7:18, 7:20, 7:35, 7:52, 7:54, 7:51]"
9050,2013,1,12581,"Symington, Michelle",F25_29,3:32:11,3:34:11,3:34:21,0:23:27,[],"[7:33, 8:11, 7:57, 7:58, 7:58, 8:09, 8:24, 8:21, 8:21, 7:49]"
9432,2013,0,9788,"Mcdonald, Joseph M",M45_49,3:33:34,3:34:46,3:53:33,0:22:28,[],"[7:14, 7:40, 7:42, 7:53, 8:57, 7:53, 8:14, 8:51, 9:12, 8:49]"
10862,2013,1,18431,"Smith, Dallas",M70_74,3:38:38,3:39:09,3:58:39,0:23:57,[],"[7:43, 7:52, 7:49, 7:57, 7:48, 8:03, 8:14, 8:39, 9:50, 9:52]"
12364,2013,0,14499,"Genetti, Anita J",F30_34,3:43:58,3:47:16,3:47:16,,"[1, 2]","[7:47, 8:07, 8:00, 8:16, 9:17, 9:44, 9:16, 9:26]"
13723,2013,1,18472,"Stepler, Joan",F45_49,3:49:22,3:50:14,4:09:24,0:25:34,[],"[8:14, 8:27, 8:30, 9:12, 8:26, 8:40, 8:59, 9:07, 9:00, 8:33]"


In [14]:
# add weather info to boscomplete: temp (F) when runners cross the starting line and halfway point
# - unofficial results have TODs
# - wunderground.com has historical weather

weather = pd.DataFrame()
for year in ['2013','2014','2015','2016','2017']:

    dfr = pd.read_csv(year+"bostonunofficial.csv",low_memory=False)

    for x in ['split1_time','split2_time','split3_time','split4_time','split5_time','split5_tod','split10_tod','chip']:
        dfr[x].replace("00:00:00","",inplace=True)
        dfr[x].replace("-","",inplace=True)
        dfr[x].replace(regex=r'\s+',value="",inplace=True)
        dfr[x].replace(np.nan,"",inplace=True)

    dfr['half_time'] = dfr.apply(lambda row: np.nan if row.split5_tod=="" else datetime.strftime(pd.to_datetime(row.split5_tod, format="%I:%M:%S%p"), "%H:%M:%S"),axis=1)
    dfr['start_tod'] = dfr.apply(lambda row: get_time(get_sec(datetime.strftime(pd.to_datetime(row.split10_tod, format="%I:%M:%S%p"), "%H:%M:%S"))-get_sec(row.chip)) if row.chip!=""
                                 else get_time(get_sec(datetime.strftime(pd.to_datetime(row.split1_tod, format="%I:%M:%S%p"), "%H:%M:%S"))-get_sec(row.split1_time)) if row.split1_time!=""
                                 else get_time(get_sec(datetime.strftime(pd.to_datetime(row.split2_tod, format="%I:%M:%S%p"), "%H:%M:%S"))-get_sec(row.split2_time)) if row.split2_time!=""
                                 else "",axis=1)
    # if start_tod is "" (unknown), then just set it to arbitrary value of '10:19:19'
    dfr['start_tod'] = dfr['start_tod'].replace("","10:19:19")
    
    # dfws: "s" for "start"
    dfws = pd.read_excel(year+"bos_weather_start.xlsx")
    dfws.columns = [x.strip() for x in dfws.columns]
    dfws.rename(columns={'Time (EDT)':'time',
                         'Temp.':'temp'},inplace=True)
    dfws.temp = dfws.apply(lambda row: float(row.temp.split()[0]),axis=1)
    dfws.time = pd.to_datetime(dfws.time)
    dfws.set_index('time',inplace=True)
    dfws = dfws.resample('1Min').interpolate()
    dfws['time'] = dfws.index
    
    # dfwh: "h" for "halfway point"
    dfwh = pd.read_excel(year+"bos_weather_start.xlsx")
    dfwh.columns = [x.strip() for x in dfwh.columns]
    dfwh.rename(columns={'Time (EDT)':'time','Temp.':'temp'},inplace=True)
    dfwh.temp = dfwh.apply(lambda row: float(row.temp.split()[0]),axis=1)
    dfwh.time = pd.to_datetime(dfwh.time)
    dfwh.set_index('time',inplace=True)
    dfwh = dfwh.resample('1Min').interpolate()
    dfwh['time'] = dfwh.index
    
    # truncate to whole minutes ("10:12:34" would get modified to "10:12:00")
    dfr.start_tod = dfr.start_tod.str.replace(r":..$",":00")
    dfr.half_time = dfr.half_time.str.replace(r":..$",":00")
    dfr['start_time'] = dfr.start_tod.apply(pd.Timestamp)
    dfr['half_time'] = dfr.half_time.apply(pd.Timestamp)
    dfr['start_temp'] = pd.merge(dfr,dfws,how='left',left_on='start_time',right_on='time')['temp']
    dfr['half_temp'] = pd.merge(dfr,dfwh,how='left',left_on='half_time',right_on='time')['temp']
    dfr['year'] = int(year)
    if year == '2013':        
        dfr.bib.replace('14995','10735',inplace=True) # Floss, Peter
        dfr.bib.replace('27141','24585',inplace=True) # Dugan, Timothy
    elif year == '2014':
        dfr.bib.replace('4994','2281',inplace=True) # Muhlemann Bk, Stefan
        dfr.bib.replace('4995','7024',inplace=True) # Miller, Justin
        dfr.bib.replace('9994','12900',inplace=True) # Schroeder, Karen
        dfr.bib.replace('35995','19257',inplace=True) # Kresky-Griffin, Lisa
        dfr.bib.replace('15992','15940',inplace=True) # Park, Hyunsoo
        dfr.bib.replace('19996','19066',inplace=True) # Holmes, Carol
    elif year == '2015':
        dfr.bib.replace('5997','5368',inplace=True) # Bell, David
        dfr.bib.replace('5994','3469',inplace=True) # Braunshausen, Andy
        dfr.bib.replace('13995','14840',inplace=True) # Dowd, Bridget
        dfr.bib.replace('5996','3136',inplace=True) # Hallgren, Krister
        dfr.bib.replace('21995','20399',inplace=True) # Harris, Rebecca
        dfr.bib.replace('13994','14381',inplace=True) # Kenutis, Sarah
        dfr.bib.replace('21994','18447',inplace=True) # Kreutz, Wendi
        dfr.bib.replace('21993','19783',inplace=True) # Marshall, Julie
        dfr.bib.replace('25994','29027',inplace=True) # Sheeleigh, Matthias
        dfr.bib.replace('5995','4411',inplace=True) # Suarez Arias, Miguel
        dfr.bib.replace('25995','24077',inplace=True) # Williamson, Delwyn
    elif year == '2016':
        dfr.bib.replace('22990','19399',inplace=True) # Altendorf, Timothy
        dfr.bib.replace('14990','8226',inplace=True) # Calderon, Ramiro
        dfr.bib.replace('15585','15144',inplace=True) # Fox, John
        dfr.bib.replace('32114','30334',inplace=True) # Francis, David
        dfr.bib.replace('4998','3945',inplace=True) # Kositzke, Jacob
        dfr.bib.replace('23599','25448',inplace=True) # Pascua, Luigi
        dfr.bib.replace('19993','19390',inplace=True) # Spence, Steve
        dfr.bib.replace('32115','31323',inplace=True) # Yagoda, Aaron
        dfr.bib.replace('19992','17209',inplace=True) # Yingling, Leah
    weather = pd.concat([weather, dfr[['year','bib','start_tod','split5_tod','start_temp','half_temp']]])
    print("finished {}!".format(year))
print("finished!")

temp = pd.merge(boscomplete, weather, how='left',on=['year','bib'])
boscomplete[['start_t','half_t']] = temp[['start_temp_y','half_temp']]

finished 2013!
finished 2014!
finished 2015!
finished 2016!
finished 2017!
finished!


In [15]:
# ADD MORE METRICS TO boscomplete

boscomplete['pace_score'] = boscomplete['chip'].apply(get_sec)*1.0/marDist - boscomplete.BQpace_sec

boscomplete['split25_score'] = (boscomplete.split5_time.apply(get_sec) - boscomplete.split2_time.apply(get_sec))*1.0/(timingmats[5]-timingmats[2]) - boscomplete.BQpace_sec
boscomplete['split35_score'] = (boscomplete.split5_time.apply(get_sec) - boscomplete.split3_time.apply(get_sec))*1.0/(timingmats[5]-timingmats[3]) - boscomplete.BQpace_sec

# calculate stdpace up until mat X
# e.g., stdpace_mat3 would be the std dev of split1_pace, split12_pace, split23_pace
for x in range(3,6):
    boscomplete["stdpace_mat" + str(x)] = boscomplete.apply(lambda row: np.nan if x not in row.non_missing_mats
        else np.std([row.non_missing_splitpaces[z] for z in range(row.non_missing_mats.index(x)+1)],ddof=1), axis=1)

for x in range(1,6):
    boscomplete['split'+str(x)] = boscomplete['split'+str(x)+'_score'] + boscomplete.BQpace_sec
    
boscomplete['split10'] = boscomplete['pace_score'] + boscomplete.BQpace_sec

boscomplete['bib_mod'] = boscomplete.bib.str.replace("F","").apply(int)
    
# add columns: split paces in seconds/mi
# e.g., split12 would be the split pace between the 1st & 2nd timing mats
for x in ['split12_score','split23_score','split34_score','split45_score','split35_score',
         'split25_score',
         'split56_score','split67_score','split78_score','split89_score','split910_score']:
    boscomplete[x.replace("_score","")] = boscomplete[x] + boscomplete.BQpace_sec

# add columns: 1st & 2nd half paces in "MM:SS" (minutes per mile)
boscomplete['pace_1sthalf'] = boscomplete['1sthalfpace'].apply(get_time)
boscomplete['pace_2ndhalf'] = boscomplete['2ndhalfpace'].apply(get_time)

/Users/donwang/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [16]:
# find people whose official chip times are slower than their unofficial chip times (rare)
df = boscomplete.copy()
cols = ['BQ','year','bib','name','DIVISION','chip','chip_unofficial','gun','mm','non_missing_splitpaces_time']
df[(df.chip>df.chip_unofficial) & (df.chip_unofficial!="")][cols] # Paul Dabene & Richard Chase

,BQ,year,bib,name,DIVISION,chip,chip_unofficial,gun,mm,non_missing_splitpaces_time
47379,0,2014,25771,"Chase, Richard L",M45_49,5:34:28,5:19:28,5:19:28,[],"[10:35, 10:22, 10:31, 11:18, 11:23, 13:13, 14:05, 15:06, 15:32, 16:44]"
128741,0,2017,28020,"Dabene, Paul Jr",M20_24,5:48:01,3:50:53,3:55:08,[7],"[48:33, 11:49, 12:29, 13:13, 12:09, 13:34, 0:15, 8:28, 8:27]"


In [17]:
# look again for anomalies, after taking care of things above

In [18]:
# find people with sub-4:20 split anywhere along course
df = boscomplete.copy()
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','stdpace','mm','non_missing_splitpaces_time']
anomaly1 = df[(df.apply(lambda row: (min(row['non_missing_splitpaces'])<=60*4.33),axis=1)==1)]
print(len(anomaly1))
(anomaly1[cols])
# it'll be best to drop these 3 altogether

3


,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time
31244,2014,0,15089,"Potts, Franzine B",F30_34,3:44:06,4:54:28,5:07:28,961.337157,[],"[7:48, 11:16, 10:24, 10:50, 9:33, 11:41, 11:46, 13:12, 12:56, -39:22]"
48209,2014,0,26680,"Martinovich, Patricia",F45_49,5:50:00,6:08:13,6:16:06,310.849490,[],"[10:05, 10:53, 11:57, 13:15, 18:14, 15:38, 15:22, 19:53, 14:30, 1:15]"
128741,2017,0,28020,"Dabene, Paul Jr",M20_24,5:48:01,3:50:53,3:55:08,809.107636,[7],"[48:33, 11:49, 12:29, 13:13, 12:09, 13:34, 0:15, 8:28, 8:27]"


In [19]:
# BQ cases (not necessarily with sub-4:20 split pace)
df = boscomplete.copy()
anomaly2 = df[df.BQ.isin([1]) 
    & (df.chip!=df.chip_unofficial)
    & (df.split1_score-df.split12_score>=30)
    & (df.split89_score-df.split910_score>=60)]
print(len(anomaly2))
(anomaly2[cols])
# These 2 from 2014 weren't in the unofficial results, and their first & last split paces look 'off'
# As we don't know how much to adjust their split times by, it's best to drop these 2 altogether

2


,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time
19707,2014,1,4632,"Warner, Joshua",M30_34,2:58:52,,,39.363109,[],"[7:41, 6:45, 6:41, 6:44, 6:40, 6:42, 6:47, 6:54, 7:06, 5:04]"
23861,2014,1,11000,"Rygg, Neisha",F35_39,3:19:14,,,22.338220,[],"[7:42, 7:06, 7:21, 7:32, 7:17, 7:35, 7:52, 8:03, 7:59, 6:56]"


In [20]:
# find chip times that don't make sense
# fix Brian Wright's chip time of 2:58:81 - convert everyone's time to seconds, then (back) to H:MM:SS
boscomplete['chip'] = boscomplete.chip.apply(get_sec).apply(get_time)
df = boscomplete.copy()
cols = ['year','BQ','bib','name','DIVISION','chip','chip_unofficial','gun','stdpace','mm','non_missing_splitpaces_time']
anomaly3 = df[(df.apply(lambda row: (int(row['chip'][-5:-3])>59)
                                          or (int(row['chip'][-2:])>59),axis=1)==1)]
(anomaly3[cols])
# Brian Wright's chip time of 2:58:81 was fixed, so it no longer appears here

,year,BQ,bib,name,DIVISION,chip,chip_unofficial,gun,stdpace,mm,non_missing_splitpaces_time


In [21]:
# find known cheaters
df = boscomplete.copy()
cols = ['year','BQ','bib','name','age','chip','stdpace','mm','non_missing_splitpaces_time']
anomaly4 = df[df.name.str.contains("(taylor, george)|(koeppe, torsten)",case=False)]\
    .sort_values(by=['age'])
print(len(anomaly4))
(anomaly4[cols])

8


/Users/donwang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,year,BQ,bib,name,age,chip,stdpace,mm,non_missing_splitpaces_time
20305,2014,1,3385,"Koeppe, Torsten",49,3:02:04,15.382833,"[7, 8]","[6:50, 6:48, 6:48, 6:53, 6:51, 7:04, 6:59, 7:34]"
52441,2015,1,2006,"Koeppe, Torsten",50,3:01:23,6.783447,"[7, 8]","[6:55, 6:49, 6:48, 6:58, 6:56, 7:07, 6:53, 7:05]"
79571,2016,1,4727,"Koeppe, Torsten",51,3:13:07,71.705190,"[7, 8]","[6:51, 6:53, 7:07, 7:32, 7:40, 9:28, 6:48, 9:47]"
7426,2013,1,8882,"Taylor, George F",59,3:26:14,58.151519,[],"[7:56, 7:02, 6:56, 8:26, 8:02, 7:13, 7:28, 7:17, 9:50, 9:12]"
28971,2014,1,14369,"Taylor, George F",60,3:36:44,81.124308,[],"[7:55, 7:55, 7:05, 7:10, 7:54, 8:25, 8:06, 7:33, 11:09, 10:25]"
61700,2015,1,19316,"Taylor, George",61,3:36:12,105.181233,[4],"[8:43, 7:34, 7:15, 7:48, 7:31, 7:15, 7:53, 10:00, 12:34]"
83567,2016,1,19749,"Taylor, George",62,3:29:06,43.292765,[4],"[8:47, 7:25, 7:15, 7:58, 7:30, 7:24, 7:54, 9:03, 8:58]"
109691,2017,1,15324,"Taylor, George",63,3:29:33,67.141966,[4],"[9:34, 7:07, 7:36, 7:12, 7:45, 7:23, 7:06, 9:46, 9:18]"


In [22]:
skip = True
if skip == False:
    print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))
    boscomplete.to_csv("BOSexample.csv",index=None)
    print(len(boscomplete))
    print(datetime.now().strftime("%A, %d %B %Y %I:%M:%S%p"))

In [23]:
# BQ cases (not necessarily with sub-4:20 split pace) -- some of these appear above in anomaly1
# split1_pace is too slow relative to split12_pace
# split89_pace is too fast relative to split910_pace
df = savethis.copy()
cols = ['year','bib','name','chip','chip_unofficial','gun','non_missing_splitpaces_time']
anomaly2 = df[df.BQ.isin([1]) 
    & (df.chip!=df.chip_unofficial)
    & (df.split1_score-df.split12_score>=30)
    & (df.split89_score-df.split910_score>=60)]
print(len(anomaly2))
(anomaly2.sort_values(by=['mm_len','stdpace'],ascending=False)[cols])
(anomaly2[cols])

46


,year,bib,name,chip,chip_unofficial,gun,non_missing_splitpaces_time
1379,2013,5599,"Ness, Erik C",2:55:59,2:59:12,2:59:12,"[7:48, 6:43, 6:37, 6:38, 6:40, 6:38, 6:46, 6:46, 6:46, 4:25]"
4267,2013,4002,"Shugart, Rob C",3:12:44,3:15:40,3:15:40,"[7:53, 6:52, 6:56, 7:01, 6:58, 8:08, 7:26, 7:53, 7:46, 5:23]"
18410,2014,2794,"Watson, Andy",2:48:22,2:49:57,2:49:57,"[7:05, 6:30, 6:23, 6:25, 6:24, 6:21, 6:26, 6:28, 6:16, 5:09]"
19454,2014,4406,"Schakel, Christiaan",2:57:29,2:59:41,2:59:41,"[7:22, 6:41, 6:40, 6:48, 6:36, 6:50, 6:52, 6:53, 6:46, 5:18]"
19497,2014,4224,"Lott, Brian",2:57:40,3:00:29,3:00:29,"[7:43, 6:44, 6:40, 6:42, 6:40, 6:42, 6:47, 6:53, 6:55, 4:46]"
19707,2014,4632,"Warner, Joshua",2:58:52,,,"[7:41, 6:45, 6:41, 6:44, 6:40, 6:42, 6:47, 6:54, 7:06, 5:04]"
19743,2014,7487,"Bertram, Troy M",2:58:59,3:04:40,3:04:40,"[8:36, 6:49, 6:39, 6:46, 6:42, 6:49, 6:52, 7:05, 6:57, 2:33]"
19996,2014,7235,"Toudal, Steen K",3:00:08,3:05:32,3:05:32,"[8:35, 6:50, 6:48, 6:53, 6:37, 6:52, 7:02, 6:58, 6:48, 2:52]"
20160,2014,5604,"Swetenburg, Raymond III",3:01:11,3:04:56,3:04:56,"[8:05, 6:51, 6:49, 6:50, 6:46, 6:52, 6:57, 7:03, 6:58, 4:22]"
20445,2014,6380,"Spouse, Antony J",3:02:52,3:07:08,3:07:08,"[8:22, 7:09, 6:44, 6:51, 6:43, 6:53, 6:57, 6:55, 7:12, 4:11]"


In [24]:
boscomplete.ix[anomaly2.index][cols]

,year,bib,name,chip,chip_unofficial,gun,non_missing_splitpaces_time
1379,2013,5599,"Ness, Erik C",2:55:59,2:59:12,2:59:12,"[6:46, 6:43, 6:37, 6:38, 6:40, 6:38, 6:46, 6:46, 6:46, 6:47]"
4267,2013,4002,"Shugart, Rob C",3:12:44,3:15:40,3:15:40,"[6:56, 6:52, 6:56, 7:01, 6:58, 8:08, 7:26, 7:53, 7:46, 7:32]"
18410,2014,2794,"Watson, Andy",2:48:22,2:49:57,2:49:57,"[6:34, 6:30, 6:23, 6:25, 6:24, 6:21, 6:26, 6:28, 6:16, 6:18]"
19454,2014,4406,"Schakel, Christiaan",2:57:29,2:59:41,2:59:41,"[6:39, 6:41, 6:40, 6:48, 6:36, 6:50, 6:52, 6:53, 6:46, 6:55]"
19497,2014,4224,"Lott, Brian",2:57:40,3:00:29,3:00:29,"[6:49, 6:44, 6:40, 6:42, 6:40, 6:42, 6:47, 6:53, 6:55, 6:50]"
19707,2014,4632,"Warner, Joshua",2:58:52,,,"[7:41, 6:45, 6:41, 6:44, 6:40, 6:42, 6:47, 6:54, 7:06, 5:04]"
19743,2014,7487,"Bertram, Troy M",2:58:59,3:04:40,3:04:40,"[6:46, 6:49, 6:39, 6:46, 6:42, 6:49, 6:52, 7:05, 6:57, 6:43]"
19996,2014,7235,"Toudal, Steen K",3:00:08,3:05:32,3:05:32,"[6:51, 6:50, 6:48, 6:53, 6:37, 6:52, 7:02, 6:58, 6:48, 6:50]"
20160,2014,5604,"Swetenburg, Raymond III",3:01:11,3:04:56,3:04:56,"[6:53, 6:51, 6:49, 6:50, 6:46, 6:52, 6:57, 7:03, 6:58, 7:07]"
20445,2014,6380,"Spouse, Antony J",3:02:52,3:07:08,3:07:08,"[7:00, 7:09, 6:44, 6:51, 6:43, 6:53, 6:57, 6:55, 7:12, 7:18]"
